In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB
from sklearn.neural_network import MLPClassifier

from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier

import os
import numpy

from algorithm_wrapper import ModelWrapper

In [3]:
# Defina os diretórios
source_dir = 'selected_images_dog_cat'

uploaded_images = []
# Percorra os arquivos no diretório de origem
for filename in os.listdir(source_dir):
    source_file = os.path.join(source_dir, filename)
    source_file = source_file.replace('\\', '/')
    uploaded_images.append(source_file)

print(uploaded_images)

['selected_images_dog_cat/Abyssinian_1.jpg', 'selected_images_dog_cat/Abyssinian_10.jpg', 'selected_images_dog_cat/Abyssinian_100.jpg', 'selected_images_dog_cat/Abyssinian_101.jpg', 'selected_images_dog_cat/Abyssinian_102.jpg', 'selected_images_dog_cat/Abyssinian_103.jpg', 'selected_images_dog_cat/Abyssinian_104.jpg', 'selected_images_dog_cat/Abyssinian_105.jpg', 'selected_images_dog_cat/Abyssinian_106.jpg', 'selected_images_dog_cat/Abyssinian_107.jpg', 'selected_images_dog_cat/Abyssinian_108.jpg', 'selected_images_dog_cat/Abyssinian_109.jpg', 'selected_images_dog_cat/Abyssinian_11.jpg', 'selected_images_dog_cat/Abyssinian_110.jpg', 'selected_images_dog_cat/Abyssinian_111.jpg', 'selected_images_dog_cat/Abyssinian_112.jpg', 'selected_images_dog_cat/Abyssinian_113.jpg', 'selected_images_dog_cat/Abyssinian_114.jpg', 'selected_images_dog_cat/Abyssinian_115.jpg', 'selected_images_dog_cat/Abyssinian_116.jpg', 'selected_images_dog_cat/Abyssinian_117.jpg', 'selected_images_dog_cat/Abyssinian_1

In [4]:
# Define a função para determinar a classe com base no nome do arquivo
def determine_class(filename):
    if 'boxer' in filename or 'wheaten_terrier' in filename:
        return 'dog'
    elif 'Siamese' in filename or 'Abyssinian' in filename:
        return 'cat'
    else:
        return 'unknown'

# Cria uma nova coluna 'Class' no dataset
classes = [determine_class(filename) for filename in uploaded_images]

# Exibe as classes
print(classes.count('dog'))
print(classes.count('cat'))
print(len(classes))
print(classes)

400
398
798
['cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 

In [5]:
import os
import json

file_metrics_names = ['bagging_models', 'bagging_models_05', 'boosting_models', 'random_forest_models', 'stacking_models', 'voting_models']

for model in file_metrics_names:
    # Define the filename
    filename = f'metrics_{model}.json'

    # Check if the file exists
    if not os.path.exists(filename):

        # Create and write the initial content to the file
        open(filename, 'w').close()

        print(f"File '{filename}' created with initial content.")
    else:
        print(f"File '{filename}' already exists.")


File 'metrics_bagging_models.json' already exists.
File 'metrics_bagging_models_05.json' already exists.
File 'metrics_boosting_models.json' already exists.
File 'metrics_random_forest_models.json' already exists.
File 'metrics_stacking_models.json' created with initial content.
File 'metrics_voting_models.json' created with initial content.


In [6]:
filename = 'features.json'

features_dict = {}

try:
    with open('features.json', 'r') as f:
        features_dict = json.load(f)
except Exception:
    print("Maybe you didn't run 'extrair_features.ipynb' first, it should be run before this one.")

In [11]:
import re

estimator_names = {
    'DecisionTreeClassifier': 'AD',
    'KNeighborsClassifier': 'k-NN',
    'GaussianNB': 'NB',
    'MLPClassifier': 'MLP',
    'StackingClassifier': 'Stacking',
    'VotingClassifier': 'Voting',
}

def run_experiments(models, model_name):
    metrics_dict = {}

    # Iterate through the dictionary to get both the name and the variable
    for feature_name, feature_data in features_dict.items():
        for model in models:

            estimators = model.get_params().get('estimators', 'default')

            criterion_name = model.get_params().get('criterion', 'default')
            print(f"Criterion name: {criterion_name}")


            estimator_name = re.match(r'(\w+)', str(model.estimator)).group(1) if hasattr(model, 'estimator') else 'None'
            estimator_name = estimator_names[estimator_name] if estimator_name in estimator_names else estimator_name
            print(f"Estimator name: {estimator_name}")


            n_estimators = model.get_params().get('n_estimators', 'default')
            print(f"Number of estimators: {n_estimators}")


            model_wrapper = ModelWrapper(model, feature_data, classes, grid_search=True)
            print(f"Training model with feature set: {feature_name}")
            name = ''
            if criterion_name == 'default':
                if estimators == 'default':
                    name = estimator_name
                else:
                    name = len(estimators)
            else:
                name = criterion_name
            model_wrapper.model_train('kfold')
            metrics_dict.update({f"{feature_name}_kfold_{name}_{n_estimators}": model_wrapper.metrics})
            model_wrapper.model_train('holdout')
            metrics_dict.update({f"{feature_name}_holdout_{name}_{n_estimators}": model_wrapper.metrics})
        

    with open(f'./metrics_{model_name}.json', 'w') as f:
        json.dump(metrics_dict, f, indent=4)

In [ ]:
bagging_models = [
    *[BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=5), n_estimators=i) for i in [10,20,30]],
    *[BaggingClassifier(estimator=KNeighborsClassifier(n_neighbors=15), n_estimators=i) for i in [10,20,30]],
    *[BaggingClassifier(estimator=GaussianNB(), n_estimators=i) for i in [10,20,30]],
    *[BaggingClassifier(estimator=MLPClassifier(activation="relu",
                                                hidden_layer_sizes=(100,50),
                                                learning_rate_init=0.0001,
                                                max_iter=500,
                                                solver="sgd"), n_estimators=i) for i in [10,20,30]],
]

run_experiments(bagging_models, 'bagging_models')

In [ ]:
bagging_models_05 = [
    *[BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=5), n_estimators=i, max_features=0.5) for i in [10,20,30]],
    *[BaggingClassifier(estimator=KNeighborsClassifier(n_neighbors=15), n_estimators=i, max_features=0.5) for i in [10,20,30]],
    *[BaggingClassifier(estimator=GaussianNB(), n_estimators=i, max_features=0.5) for i in [10,20,30]],
    *[BaggingClassifier(estimator=MLPClassifier(activation="relu",
                                                hidden_layer_sizes=(100,50),
                                                learning_rate_init=0.0001,
                                                max_iter=500,
                                                solver="sgd"), n_estimators=i, max_features=0.5) for i in [10,20,30]],
]

run_experiments(bagging_models_05, 'bagging_models_05')

In [ ]:
boosting_models = [
    *[AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=5), n_estimators=i) for i in [10,20,30]],
    *[AdaBoostClassifier(estimator=GaussianNB(), n_estimators=i) for i in [10,20,30]]
]

run_experiments(boosting_models, 'boosting_models')

In [ ]:
random_forest_models = [
    *[RandomForestClassifier(criterion='gini', n_estimators=i) for i in [10,20,30,100]],
    *[RandomForestClassifier(criterion='entropy', n_estimators=i) for i in [10,20,30,100]],
    *[RandomForestClassifier(criterion='log_loss', n_estimators=i) for i in [10,20,30,100]],
]

run_experiments(random_forest_models, 'random_forest_models')

In [15]:
def choose_estimators():
    final_estimators = []

    mlp_classifiers = [(f'MLP_{i}', MLPClassifier(activation="relu",
                                                hidden_layer_sizes=i,
                                                learning_rate_init=0.0001,
                                                max_iter=500,
                                                solver="adam")) for i in [10,20,30]]
    
    knn_classifiers = [(f'kNN_{i}',KNeighborsClassifier(n_neighbors=i)) for i in range(5, 15)]

    ad_classifiers = [(f'AD_{i}', DecisionTreeClassifier(max_depth=i)) for i in range(2, 10)]

    nb_classifiers = [('NB', GaussianNB())]

    classifiers = mlp_classifiers + knn_classifiers + ad_classifiers + nb_classifiers
    numpy.random.shuffle(classifiers)


    for i in range (1, 5):
        
        chosen_estimators = classifiers[:i*5]
        # for _ in range (1, i * 5 + 1):
        #     mlp_params = {
        #         'activation': 'relu',
        #         'learning_rate_init': 0.001,
        #         'hidden_layer_sizes': numpy.random.choice(range(10, 50, 10)),
        #         'max_iter': 500,
        #         'solver': numpy.random.choice(['adam', 'sgd'])
        #     }

        #     knn_params = {
        #         'n_neighbors': numpy.random.choice(range(5, 20))
        #     }

        #     ad_params = {
        #         'max_depth': numpy.random.choice(range(1, 15))
        #     }

        #     estimators = [('MLP', MLPClassifier(**mlp_params)),
        #                 ('kNN', KNeighborsClassifier(**knn_params)),
        #                 ('NB', GaussianNB()),
        #                 ('AD', DecisionTreeClassifier(**ad_params))]
            
        #     chosen_estimators.append(numpy.random.choice(estimators))
        
        final_estimators.append(chosen_estimators)
    
    print(final_estimators)
    return final_estimators

In [16]:
from sklearn.linear_model import LogisticRegression

final_estimators = choose_estimators()
        
stacking_models = [
    *[StackingClassifier(estimators=estimators, final_estimator=LogisticRegression()) for estimators in final_estimators]
]

run_experiments(stacking_models, 'stacking_models')

[[('NB', GaussianNB()), ('kNN_8', KNeighborsClassifier(n_neighbors=8)), ('AD_8', DecisionTreeClassifier(max_depth=8)), ('kNN_14', KNeighborsClassifier(n_neighbors=14)), ('MLP_20', MLPClassifier(hidden_layer_sizes=20, learning_rate_init=0.0001, max_iter=500))], [('NB', GaussianNB()), ('kNN_8', KNeighborsClassifier(n_neighbors=8)), ('AD_8', DecisionTreeClassifier(max_depth=8)), ('kNN_14', KNeighborsClassifier(n_neighbors=14)), ('MLP_20', MLPClassifier(hidden_layer_sizes=20, learning_rate_init=0.0001, max_iter=500)), ('kNN_5', KNeighborsClassifier()), ('kNN_7', KNeighborsClassifier(n_neighbors=7)), ('kNN_10', KNeighborsClassifier(n_neighbors=10)), ('kNN_13', KNeighborsClassifier(n_neighbors=13)), ('AD_2', DecisionTreeClassifier(max_depth=2))], [('NB', GaussianNB()), ('kNN_8', KNeighborsClassifier(n_neighbors=8)), ('AD_8', DecisionTreeClassifier(max_depth=8)), ('kNN_14', KNeighborsClassifier(n_neighbors=14)), ('MLP_20', MLPClassifier(hidden_layer_sizes=20, learning_rate_init=0.0001, max_it

In [ ]:
final_estimators = choose_estimators()
        
stacking_models = [
    *[VotingClassifier(estimators=estimators, voting='hard') for estimators in final_estimators]
]

run_experiments(stacking_models, 'stacking_models')